In [1]:
import nltk
nltk.download('wordnet')

import pandas as pd
import io, requests

movies = pd.read_csv("movies_metadata.csv", usecols=['original_title', 'overview', 'title'])
movies = movies.dropna(axis=0)
movies.shape

[nltk_data] Downloading package wordnet to /Users/masterp/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


(44506, 3)

In [2]:
movie_plot_li = movies['overview']
movie_info_li = movies['title']
movies.head(3)

,original_title,overview,title
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Toy Story
1,Jumanji,When siblings Judy and Peter discover an encha...,Jumanji
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,Grumpier Old Men


In [4]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
        self.tokenizer = RegexpTokenizer('(?u)[A-z]+')
    def __call__(self, doc):    #클래스 호출시 마다 실행(Tf-idf Vector 호출)
        return([self.wnl.lemmatize(t) for t in self.tokenizer.tokenize(doc)])

In [7]:
#사이킷런에 위에서 정의한 토크나이저를 입력으로 넣습니다.

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=3, tokenizer=LemmaTokenizer(), stop_words='english')
X = vectorizer.fit_transform(movie_plot_li[:10000])
vocabluary = vectorizer.get_feature_names()

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [8]:
#비슷한 영화 추천하는 Cosin 유사모델 만들기

from sklearn.metrics.pairwise import cosine_similarity

movie_sim = cosine_similarity(X)
print(movie_sim.shape)
movie_sim

(10000, 10000)


array([[1.        , 0.02930725, 0.00580921, ..., 0.01088994, 0.01071711,
        0.        ],
       [0.02930725, 1.        , 0.05286611, ..., 0.00904565, 0.00890209,
        0.        ],
       [0.00580921, 0.05286611, 1.        , ..., 0.00466609, 0.00459204,
        0.        ],
       ...,
       [0.01088994, 0.00904565, 0.00466609, ..., 1.        , 0.00860824,
        0.        ],
       [0.01071711, 0.00890209, 0.00459204, ..., 0.00860824, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [18]:
#특정 영화와 유사한 영화목록 출력하기

def similar_recommend_by_movie_id(movielens_id, rank=8):
    movie_index = movielens_id -1
    similar_movies = sorted(list(enumerate(movie_sim[movie_index])), key=lambda x:x[1], reverse=True)
    
    print("----- {} : 관람객 추천영화 -----".format(movie_info_li[similar_movies[0][0]]))
    
    for no, movie_idx in enumerate(similar_movies[1:rank]):
        print('추천영화 {}순위 : {}'.format(no, movie_info_li[movie_idx[0]]))

In [24]:
similar_recommend_by_movie_id(8, rank=15)

----- Tom and Huck : 관람객 추천영화 -----
추천영화 0순위 : Son in Law
추천영화 1순위 : Heavy Metal
추천영화 2순위 : Cecil B. Demented
추천영화 3순위 : The Great Ziegfeld
추천영화 4순위 : A Guy Thing
추천영화 5순위 : Regeneration
추천영화 6순위 : The Year of Living Dangerously
추천영화 7순위 : The Black Scorpion
추천영화 8순위 : DragonHeart
추천영화 9순위 : Frankie and Johnny
추천영화 10순위 : Lethal Weapon 2
추천영화 11순위 : Bagdad Cafe
추천영화 12순위 : Audition
추천영화 13순위 : Rhyme & Reason
